This section seeks to provide a ranking of how efficiently teams spend their money on a yearly basis. These values will be computed by using a model to predict the salary of a given player on a given team, in a given year. This salary will be compared with the player's actual salary and the difference between the actual and the predicted will tell us if someone is overpaid or underpaid. We will then sum all of these values up for a given team and this will tell us the total amount that they are overpaying or underpaying by. We can then divide this by the total amount spent on salaries, so that we can see overpay or underpay as a percentage of total salary spent, which should make it easier to compare these values across teams and across time.

In [3]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Load in neural network for prediction
from common import *
nn = get_nn()
nba_data = get_baseline_data("../data/cleaned_data/base_cleaned.csv")
# nba_data.loc[nba_data['playerName'] == 'Michael Jordan'][['playerName','seasonStartYear', 'team']]


Index(['playerName', 'seasonStartYear', 'salary', 'inflationAdjSalary',
       'startYear', 'position', 'height', 'weight', 'MP', 'PTS', 'Age', 'team',
       'games', 'games_started', 'PER', 'FTr', 'AST', 'STL', 'TRB', 'FT', '3P',
       'FG', 'inflationAdjSalary_log'],
      dtype='object')


c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['inflationAdjSalary_log'] = nba['inflationAdjSalary'].apply(lambda x: np.log(x))


,playerName,seasonStartYear,team
0,Michael Jordan,1996,CHI
415,Michael Jordan,1997,CHI
3023,Michael Jordan,2002,WAS


In [38]:
# Compute the difference between the amount a team is predicted to spend v. the amount that they actually spent
COL_START_YEAR = 'seasonStartYear'
COL_TEAM = 'team'
COL_PLAYER_NAME = 'playerName'
COL_SALARY = 'inflationAdjSalary'
for year in pd.unique(nba_data[COL_START_YEAR]) :
    for team in pd.unique(nba_data.loc[nba_data[COL_START_YEAR] == year][COL_TEAM]) :
        team_year_player_data = nba_data.loc[(nba_data[COL_TEAM] == team) & (nba_data[COL_START_YEAR] == year)]
        actual_pred_dif = 0
        total_salary_spend = 0
        pred_total_salary_spend = 0
        for i, player in team_year_player_data.iterrows() :
            total_salary_spend += player[COL_SALARY]
            x = (player[get_nn_features()].values).reshape(1,-1)
            pred_total_salary_spend += np.exp(nn.predict(x))
            print(np.exp(nn.predict(x)))
            actual_pred_dif += player[COL_SALARY] - np.exp(nn.predict(x))
        print("In ", year, " team ", team, " spent ", total_salary_spend, " but should have only spent ", pred_total_salary_spend)
        print("Dif: ", total_salary_spend - pred_total_salary_spend)
    break


[1.684315]
[6.88058536]
[3.71224376]
[3.00105869]
[2.94168864]
[2.3403898]
[2.26565539]
[2.93798131]
[1.18036752]
[2.10615265]
[2.80909559]
In  1996  team  CHI  spent  92432116  but should have only spent  [31.85953372]
Dif:  [92432084.14046629]
[2.69001969]
[2.48869053]
[2.98495459]
[5.2715197]
[3.04949803]
[2.61819671]
[2.95388912]
[2.20549]
[4.01508912]
[2.18098234]
[2.67340365]
In  1996  team  ORL  spent  83610754  but should have only spent  [33.13173347]
Dif:  [83610720.86826654]
[2.805772]
[3.99719597]
[3.31210389]
[2.53446439]
[4.00413468]
[11.18857834]
[2.53793477]
[2.58351366]
[1.91673578]
[2.61536229]
[3.31402199]
[2.46483329]
[1.86112079]
In  1996  team  IND  spent  67561617  but should have only spent  [45.13577184]
Dif:  [67561571.86422816]
[3.30288383]
[2.82070437]
[1.80742499]
[2.69201161]
[2.80007712]
[3.31125211]
[3.24890664]
[2.83835501]
[2.53134652]
[3.14482028]
[2.10137992]
[2.52442065]
[2.17088323]
In  1996  team  SEA  spent  52453620  but should have only spent  